<a href="https://colab.research.google.com/github/snehsagarajput/darknet/blob/master/Face_Mask-Social%20Distancing-YOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check if NVIDIA GPU is enabled
#hope you get Nvida Tesla P100 (or, Tesla T-4) (best performance for colab)
!nvidia-smi

In [ ]:
#Mounting Drive(use gmail-id having dataset) 
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive #important to set this hard link to avoid problem due to "My Drive" having space in the directory path
!ls /mydrive #content of mounted drive 

In [ ]:
#############################################################################################################
##    cloning my github with forked and updated darknet repository(u can fork this for your purpose :))    ##
##    i have added:-                                                                                       ##
##       1. config file (for training and testing:::change according to your usage)                        ##
##       2. obj.names file (class names for classification)                                                ##
##       3. obj.data file (having details of directory)                                                    ##
#############################################################################################################

!git clone https://github.com/snehsagarajput/darknet

In [ ]:
#change makefile to have GPU and OPENCV enabled (will take time)
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [ ]:
# Create folder on google drive so that we can save weights there (ignore if already exists)
!mkdir "/mydrive/YOLOv4"

In [ ]:
# Download weights darknet model 137 in GDRIVE ###ignore if already downloaded (you don't need to explicitly download this cell will do the work)
weights_dir="/content/gdrive/My Drive/YOLOv4/yolov4.conv.137" #weights location
import os.path
from os import path
if not path.exists(weights_dir):
  !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137 -P "/content/gdrive/My Drive/YOLOv4"
else:
  print("File already exists!!!!!!")

In [ ]:
#copy training images(dataset) and labels{both in one .zip file} from GDRIVE to darknet/data/obj folder
import os
gdrive_image_source_path="/content/gdrive/My Drive/YOLOv4/data/" #(address of folder containing image and annotations)

%cd /content/darknet/data/
!mkdir obj  #create obj folder as it does not already exists in data
!unzip "/content/gdrive/My Drive/YOLOv4/data/train.zip" -d "/content/darknet/data/obj/"

In [ ]:
import os
files=os.listdir("/content/darknet/data/obj")
print("No of images for training : ",len(files)//2)

In [ ]:
#copy validation images(dataset) and labels{both in one .zip file} from GDRIVE to darknet/data/obj folder
import os
gdrive_image_source_path="/content/gdrive/My Drive/YOLOv4/data/" #(address of folder containing image and annotations as .zip file)

%cd /content/darknet/data/
!unzip "/content/gdrive/My Drive/YOLOv4/data/valid.zip" -d "/content/darknet/data/obj/"

In [ ]:
#copy train.txt and valid.txt files having image and annotations location to 'data' directory in darknet
!cp "/content/gdrive/My Drive/YOLOv4/data/train.txt" "/content/darknet/data"
!cp "/content/gdrive/My Drive/YOLOv4/data/valid.txt" "/content/darknet/data"

#Define Helper Functions

These three functions are helper functions that will allow you to show the image in your Colab Notebook after running your detections, as well as upload and download images to and from your Cloud VM.

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# Training

In [ ]:
# >6000 iterations
!rm "/content/darknet/cfg/yolov4_training.cfg"
!cp "/content/gdrive/My Drive/YOLOv4/data/yolov4_training.cfg" "/content/darknet/cfg"

In [ ]:
# Start the training (set up weight)
%cd /content/darknet
!./darknet detector train data/obj.data cfg/yolov4_training.cfg "/content/gdrive/My Drive/YOLOv4/yolov4_training_last.weights" -dont_show -map

In [ ]:
#testing on video file
%cd /content/darknet
!./darknet detector demo data/obj.data cfg/yolov4_testing.cfg "/content/gdrive/My Drive/YOLOv4/yolov4_training_best.weights" "/content/gdrive/My Drive/test_street_main.mp4" -out_filename "/content/gdrive/My Drive/main_street_20000.mp4" -dont_show

In [ ]:
#for testing on single image
%cd /content/darknet
!./darknet detector test data/obj.data cfg/yolov4_testing.cfg "/content/gdrive/My Drive/YOLOv4/yolov4_training_20000.weights" "/content/gdrive/My Drive/2_class only_YOLOv4/test/222.jpg" -dont_show -map
imShow('predictions.jpg')

In [ ]:
#metrics calculation on weights
%cd /content/darknet
!./darknet detector map data/obj.data cfg/yolov4_testing.cfg "/content/gdrive/My Drive/YOLOv4/yolov4_training_last.weights"